# 極性判定とDoc２Vecを使ったTwitterネガポジ予測
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
### 【このnotebookについて】
2019年7〜10月までフルタイムで通っていたスクールの卒業課題テーマを、機械学習の勉強のために発展させたものです<br>
卒業発表スライド　https://www.slideshare.net/secret/y0m7g1nZdxpVYP<br>
＊当初は炎上予測がテーマだったので、このnotebookの内容とはややズレます<br>
＊表紙スライドの字が見えない場合は２枚目から戻ると見えます<br>

＊ちなみに…<br>
スクールで取り組んだ課題のリポジトリ 
https://github.com/kaorisugi/diveintocode-ml<br>
論文読解課題のスライドシェア 
https://www.slideshare.net/secret/qGmdiwl4uGS20O<br>

### 【ゴール】
これからツイートする予定の文章に対し、過去の類似ツイートを探し、反応のネガポジスコア付きで上位１０位まで提示する。<br>
### 【モデルの仕組み】
１）ツイートデータセットを取得<br>
　・TwitterAPIを使ってツイートを取得<br>
　・各ツイートに対する反応ツイート（リプライ、引用RT）を取得<br>
　・反応ツイートの極性表現数をカウントしてネガポジスコアとpositive/negative/fire!!!判定を得る<br>
 　（positive/negativeの判定基準：極性表現数が多い方、fire!!!(炎上）の判定基準：極性表現の７０％以上がnegative）
２）データセットの前処理<br>
　・正規表現、ストップワード除去など<br>
３）予測モデルを生成<br>
　・データセットをDoc２vecで学習<br>
４）ツイート予定文章のネガポジ予測を返す<br>
　・データセットから、ツイート予定文書と似ている文書を探す<br>
 ・ネガポジスコア付きで、類似ツイート上位１０個を返す
### 【結果】
類似度確認用にデータセット内にあるものと同じ文を入力したところ、類似度1位で返ってきた。また、２位、３位にもマスクに関する似た話題のツイートが提示されたので類似ツイートの抽出は成功。ネガポジスコアもデータズレなどなく正確に表示され、目的は達成できた。<br>
ツイッターAPI制限により、まだサンプルが少ない（完成時２００件程度）が、データを蓄積できる仕様にしているので、ツイート文のバリエーションを増やしていけば、様々な入力文に対応できるようになると思う。<br>
ネガポジ判定については、ネガティブなテーマへの言及に共感したコメントでネガ判定が出ているケースも多く、必ずしもツイート主へのネガ感情ではないことに注意が必要。<br>

### 【その他試みたこと】
１）文章ベクトルを特徴量としたネガポジ予測モデル<br>
　・文章ベクトルとフォロワー数を特徴量X、ネガポジスコアを目的変数yとしたデータを学習<br>
　・文章ベクトルはDoc２vecとTf-idfの２種を作成<br>
　・ツイート予定文書を入力してネガポジスコアを予測する<br>
　・試した予測モデル<br>
　・MultiOutputRegressor、SVRのrbf と　SVRの線形、lightgbm、ランダムフォレスト<br>
　  　→精度が低すぎて断念<br>
２）ツイッターAPI制限への挑戦（データセットの拡大）<br>
　・古いツイートを大量取得できるパッケージを発見（通常は１週間程度しか遡れない）<br>
　　　→取得データから反応ツイートの取得を試みたができなかった<br>
   
### 【利用するには】
・config.py ファイルにツイッターAPIトークンを記入<br>
・インストールが必要なツールは、notebook内にマジックコマンドにて記載してあります<br>
・import MeCab で詰まる場合は、MeCabバインディングのnatto-pyからimportするとうまくいきそうです。<br>
（$ pip install natto-pyでインストールした上でfrom natto import MeCab) <br>
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝

## １）ツイートデータセットを取得
・TwitterAPIでツイートを取得<br>
・各ツイートに対するリプライ、引用RTを取得<br>
・極性表現数をカウントしてネガポジスコアを得る<br>

#### 参考サイト
【Python】tweepyでTwitterのツイートを検索して取得<br>
https://vatchlog.com/tweepy-search/<br>
【Python】tweepyで期間指定してツイートを検索する<br>
https://vatchlog.com/tweepy-search-time/<br>
バズったツイートへのリアクションを感情分析してみる<br>【Google Natural Language API / Python】<br>
https://qiita.com/matsuri0828/items/029b4d0d510dcfb5c5dd

In [1]:
#必要なツールをインストール（初回のみ実行）
! pip install --upgrade pip
! pip install tweepy
! pip install oseti
! pip install requests requests_oauthlib
! pip install sengiri

Requirement already up-to-date: pip in /opt/conda/lib/python3.6/site-packages (19.3.1)


In [3]:
import tweepy
import re
import emoji
import oseti
from datetime import datetime, date, timedelta
import os
import pandas as pd
import csv
from tqdm import tqdm
import config

class Get_Twitter():

    def __init__(self, day, reload, print_rep = False, exclud_words = "配信スタート ＃キャンペーン　リツイートキャンペーン", RT_count = 5000):
        self.oseti_analyzer = oseti.Analyzer()  #極性判定
        self.CK = config.CONSUMER_KEY
        self.CS = config.CONSUMER_SECRET
        self.AT = config.ACCESS_TOKEN
        self.AS = config.ACCESS_TOKEN_SECRET
        self.ew = exclud_words
        self.print_rep = print_rep
        self.rt = str(RT_count)
        self.columns = [
            "Id", "Date", "Name", "Full_text", 
            "Judge", "Posi_score", "Nega_score", "Followers", "link"
        ]
        self.posi_pd = pd.DataFrame([], columns = self.columns)
        self.nega_pd = pd.DataFrame([], columns = self.columns)
        self.fire_pd = pd.DataFrame([], columns = self.columns)
        self.wait = 0
        self.reload = reload
        day = datetime.strptime(day, '%Y-%m-%d')
        self.day = day.strftime('%Y-%m-%d')

    def main(self):
        self._Make_Dir() # データ格納ファイルの準備

        #ツイートを取得、センチメント判定
        try:
            status = self.Get_Buzz() #バズったツイート取得
            for i in status:            
                if self.wait == 10:
                    print("10回待機したため終了")
                    break
                self.Status(i)
                if self.Exclude_Word(self.buzz_full_text) == True:# 除外ワードを含むツイートは除外
                    continue
                if self.Text_Count() == True: #30W以下のツイートは除外
                    continue
                self.Get_Rep() #リプライを取得
                self.Get_RT() #RTコメントを取得
                if self.Min_Rep() == False: # コメントが少ないツイートは除外
                    continue
                self.Get_Senti() #コメントをセンチメント判定
                self._Get_Analysis() #ツイートをセンチメント判定
        #エラー時はスキップして次のツイート取得
        except (ValueError,  KeyError, TypeError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            if self.reload:
                self.wait += 1
                print("==========")
                print('get_buzzのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass
        
        #生成したデータをprint
        print()
        print("↓↓↓positiveサンプル↓↓↓")
        display(self.posi_pd.head())
        print()            
        print("↓↓↓negativeサンプル↓↓↓")
        display(self.nega_pd.head())
        print()
        print("↓↓↓fire_tweetサンプル↓↓↓")
        display(self.fire_pd.head())
        print()
        print()
        
        #生成したPandasDataFrameをcsvで書き出す
        total_pd = pd.concat([self.posi_pd, self.nega_pd, self.fire_pd], ignore_index=True)
        buzz_old = pd.read_csv('./output/buzz_tweet.csv')
        buzz_new = pd.concat([buzz_old, total_pd])#既存データと連結
        buzz_new.drop_duplicates(subset="Id",inplace=True)#重複ID行を削除            
        buzz_new.to_csv('./output/buzz_tweet.csv', index = False, header = True)
        print("csvへの書き出しが完了しました。新規データ数{}、全データ数：{}".format(len(buzz_new) - len(buzz_old), len(buzz_new)))
        print("サンプルが0件の場合は、15分後に再度実行すると取得できる場合があります。") 
        print("fire_tweetは出現率が非常に低いです。")

    #Api認証
    def _Auth(self):
        auth = tweepy.OAuthHandler(self.CK, self.CS)
        auth.set_access_token(self.AT, self.AS)
        api = tweepy.API(auth)
        return api

    #出力用ディレクトリとcsvファイルを作成（存在しない場合のみ）
    def _Make_Dir(self):
        new_dir_path = 'output'
        try:
            os.makedirs(new_dir_path)
        except FileExistsError:
            pass
        if (os.path.isfile('./output/buzz_tweet.csv')) == False:
            self.posi_pd.to_csv('./output/buzz_tweet.csv', index = False)  

    #絵文字削除
    def _remove_emoji(self, text):
        return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)

    #テキストを正規表現処理、絵文字削除
    def _format_text(self, text):
        text=re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
        text=re.sub('\n', "", text)
        text=re.sub(r'@?[!-~]+', "", text)
        text=self._remove_emoji(text)
        return text
    
    #　日付表記を整える、日本時間に修正
    def _date_format(self, date):
        date = datetime.strptime(str(date), '%a %b %d %H:%M:%S %z %Y')
        date = date + timedelta(hours=9)
        return datetime.strftime(date, '%Y-%m-%d %H:%M')

    def Status(self, status): 
        self.buzz_id = status._json['id']
        self.buzz_id_str = status._json['id_str']
        self.buzz_name = status._json['user']['screen_name']
        self.buzz_full_text = status._json['full_text']
        self.date = status._json['created_at']
        self.date = self._date_format(self.date)
        self.favo = status._json['favorite_count']
        self.rt_count = status._json['retweet_count']
        api = self._Auth()
        self.followers = status._json['user']['followers_count']
        #self.followers = len(api.followers(status._json['user']['screen_name']))
    
    #除外ワード
    def Exclude_Word(self, text):                        
        if self.ew in str(text):
            print("==========")
            print("除外ワード")
            print()
            return True
        else:
            return False

    #ツイート内にリンクがあれば分割
    def Text_Count(self):
        if re.search("(https://t.co/\w+)", self.buzz_full_text) == None:
            self.link = None
        else:                   
            self.buzz_full_text = re.split("(https://t.co/\w+)", self.buzz_full_text)
            self.link = self.buzz_full_text[1]
            self.buzz_full_text = self.buzz_full_text[0]
        if len(self.buzz_full_text) < 30:
            return True

    #リプライ＋引用RTコメントが100未満のツイートは除外
    def Min_Rep(self):
        reply_texts_rows = []
        if self.rep_cnt + self.RTcomme_cnt > 100:
            reply_texts_rows.append(self.rep_row)
            reply_texts_rows.append(self.rt_row)
            return True
        else:
            return False

    #sentiment_listを一次元にし、ツイートごとの極性表現の総和の辞書にする
    def Get_Senti(self):
        self.sentiment_list = sum(self.sentiment_list, [])#１次元にする
        self.sentiment = dict((key, sum(d[key] for d in self.sentiment_list)) for key in self.sentiment_list[0])

    #バズったツイートを取得(デフォルト：5000RT以上)
    def Get_Buzz(self):
        api = self._Auth()
        try:       
            status = api.search(q = 'filter:safe min_retweets:' + self.rt + ' exclude:retweets until:' + self.day,
                lang ='ja', count =100, tweet_mode = 'extended', result_type = 'recent')
            return status
        #エラー時はスキップして次のツイート取得
        except (ValueError,  KeyError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except (tweepy.RateLimitError, tweepy.TweepError) as e:
            if self.reload:
                self.wait += 1
                print("==========")
                print('get_buzzのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass
        #return status
    
    #リプライを取得
    def Get_Rep(self):
        api = self._Auth()     
        query_reply = '@' + self.buzz_name + ' exclude:retweets'
        self.rep_row = []
        self.sentiment_list = []
        self.rep_cnt =0
        wait_cnt = 0
        try:
            for status_reply in api.search(q=query_reply, lang='ja', count=100):
                if status_reply._json['in_reply_to_status_id_str'] == self.buzz_id_str:
                    row = self._format_text(status_reply._json['text'])
                    #極性判定
                    sentiment_score = self.oseti_analyzer.count_polarity(str(row))#strにする
                    self.sentiment_list.append(sentiment_score)
                    self.rep_row.append(row)
                    self.rep_cnt += 1
                else:
                    pass
        #エラーはスキップして次のツイート取得
        except (ValueError,  KeyError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            self.wait += 1
            if self.reload:
                print("==========")
                print('get_repのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass

    # 引用RTを取得
    def Get_RT(self):
        api = self._Auth()
        query_quote = self.buzz_id_str + ' exclude:retweets'
        self.RTcomme_cnt = 0
        self.rt_row = []
        try:
            for status_quote in api.search(q=query_quote, lang='ja', count=100):
                if status_quote._json['id_str'] == self.buzz_id_str:
                    continue
                else:
                    row = self._format_text(status_quote._json['text'])
                #極性判定
                sentiment_score = self.oseti_analyzer.count_polarity(str(row))#strにする
                self.sentiment_list.append(sentiment_score)
                self.rt_row.append(row)
                self.RTcomme_cnt += 1
        #エラーはスキップして次のツイート取得
        except (ValueError,  KeyError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            self.wait += 1
            if self.reload:
                print("==========")
                print('get_rtのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass        

    #取得したTweetをprint
    def _Print(self):
        print("name：", self.buzz_name, "／フォロワー数：", self.followers)
        print("date：", self.date, "／ツイートID：", self.buzz_id_str)
        print("RT数：", self.rt_count, "／favorite数：", self.favo)
        print("リプライ数：", self.rep_cnt, "／RTコメント数(上限１００）：", self.RTcomme_cnt)
        if self.print_rep == True:
            print("リプライ\n", self.rep_row)
            print("RTコメント\n", self.rt_row)
        else:
            pass

    #センチメント判定結果を取得
    def _Get_Analysis(self):
        total = self.sentiment["positive"] + self.sentiment["negative"]
        if self.sentiment["positive"] >= self.sentiment["negative"]:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:positive】　　極性表現数", self.sentiment)
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "positive", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.posi_pd = self.posi_pd.append(s, ignore_index=True)
        elif self.sentiment["negative"]/total >= 0.7:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:fire!!!】　　極性表現数", self.sentiment)
            print("ネガ表現の割合{:.3g}".format(self.sentiment["negative"]/total))
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "fire", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.fire_pd = self.fire_pd.append(s, ignore_index=True)
        else:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:negative】　　極性表現数", self.sentiment)
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "negative", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.nega_pd = self.nega_pd.append(s, ignore_index=True)
        print()



### ツイートデータセット取得　実行

In [4]:
# 指定日のツイートを取得（API制限のため取得できるのは約一週間前のものまで）
day = '2019-12-28'
# リクエスト制限対応：True:リクエスト上限に達したら15分待機ののちツイート取得続行/ False:待機せずcsv取得
reload = True

#除外ワード
exclud_words = "配信スタート ＃キャンペーン　リツイートキャンペーン WWWWWWWWW"

#その他設定可能パラメータ
#リプライをprint（print_rep = True/Fals), 最低RT数(RT_count = 5000)

GT = Get_Twitter(day, reload, exclud_words)
GT.main()

【鬼滅の刃コラボ中！】
ローソン国際展示場駅前店では1日限定で「鬼滅の刃」コラボを実施中です！

キャラクタースタンディやポスターの展示、また入店音が炭治郎・禰豆子・善逸・伊之助のボイス（ランダム）となっておりますのでぜひチェックしてください！

#鬼滅の刃 

【判定:positive】　　極性表現数 {'positive': 20, 'negative': 16}
name： kimetsu_off ／フォロワー数： 959187
date： 2019-12-28 08:56 ／ツイートID： 1210710961771859968
RT数： 6367 ／favorite数： 34919
リプライ数： 3 ／RTコメント数(上限１００）： 99

朝から気分悪くしてごめんなさい。
はっきり申し上げます。
飲み屋のモラルやマナーを知らない人が本当に多すぎる。
文句があるなら来なくて大丈夫です。
私たちはボランティアでもなんでもないので。
年明けからこんなこと言いたくないから
年明ける前に過去のまとめたのを置いておきます。 

【判定:positive】　　極性表現数 {'positive': 182, 'negative': 130}
name： takuya_hyon ／フォロワー数： 662586
date： 2019-12-28 08:48 ／ツイートID： 1210709023726526464
RT数： 13142 ／favorite数： 53839
リプライ数： 72 ／RTコメント数(上限１００）： 100

若月健矢選手、立花理香さん

ご結婚おめでとうございます！
末長くお幸せに✨

バファローズ㊗️ポンタ

【判定:positive】　　極性表現数 {'positive': 67, 'negative': 3}
name： bs_ponta ／フォロワー数： 288626
date： 2019-12-28 08:34 ／ツイートID： 1210705650491113472
RT数： 6691 ／favorite数： 12497
リプライ数： 58 ／RTコメント数(上限１００）： 48

ONE PIECEと僕たち嵐がコラボレーションさせていただいた、新曲「A-RA-SHI：Reborn」のミュージックビデオは1週間後にリリース！
The "

「鬼の手」袋を編みました。

・左手にジャストフィット！
・厚手の生地であったかい！
・妖怪退治にも使える！
・今冬のマストアイテム！ 

【判定:positive】　　極性表現数 {'positive': 70, 'negative': 22}
name： bon_66 ／フォロワー数： 1271
date： 2019-12-27 19:41 ／ツイートID： 1210511013507784705
RT数： 23397 ／favorite数： 57095
リプライ数： 81 ／RTコメント数(上限１００）： 100

「刀剣乱舞 大演練」2020年8月 東京ドームにて開催決定！


【判定:positive】　　極性表現数 {'positive': 50, 'negative': 44}
name： stage_touken ／フォロワー数： 251499
date： 2019-12-27 19:17 ／ツイートID： 1210505051136856064
RT数： 15045 ／favorite数： 23168
リプライ数： 43 ／RTコメント数(上限１００）： 100

先輩がうざい後輩の話を描きました。【97】

#先輩がうざい後輩の話 

【判定:positive】　　極性表現数 {'positive': 56, 'negative': 23}
name： shiromanta1020 ／フォロワー数： 768166
date： 2019-12-27 19:17 ／ツイートID： 1210505003737026563
RT数： 19628 ／favorite数： 86393
リプライ数： 21 ／RTコメント数(上限１００）： 100

コミックマーケットでは深夜来場は認めておりません。絶対に深夜来場をしないようにお願いします。すみやかにお帰り下さい。 #C97 

【判定:negative】　　極性表現数 {'positive': 70, 'negative': 99}
name： comiketofficial ／フォロワー数： 221336
date： 2019-12-27 19:15 ／ツイートID： 1210504343675260930
RT数： 5492 ／favorite数： 4704
リプライ数： 3 ／RTコメント数(上限１００

,Id,Date,Name,Full_text,Judge,Posi_score,Nega_score,Followers,link
0,1210710961771859968,2019-12-28 08:56,kimetsu_off,【鬼滅の刃コラボ中！】\nローソン国際展示場駅前店では1日限定で「鬼滅の刃」コラボを実施中で...,positive,20,16,959187,https://t.co/TgagYYCcAU
1,1210709023726526464,2019-12-28 08:48,takuya_hyon,朝から気分悪くしてごめんなさい。\nはっきり申し上げます。\n飲み屋のモラルやマナーを知らな...,positive,182,130,662586,https://t.co/BeaCWjpNg4
2,1210705650491113472,2019-12-28 08:34,bs_ponta,若月健矢選手、立花理香さん\n\nご結婚おめでとうございます！\n末長くお幸せに✨\n\nバ...,positive,67,3,288626,None
3,1210703193405046786,2019-12-28 08:25,arashi5official,ONE PIECEと僕たち嵐がコラボレーションさせていただいた、新曲「A-RA-SHI：Re...,positive,118,36,2221569,https://t.co/hHobubpb16
4,1210702646509588480,2019-12-28 08:22,RiccaTachibana,【ご報告】この度、みなさまにご報告したいことができました。ぜひご覧いただけますと幸いです。,positive,117,13,175131,https://t.co/hR8m5IHoBD



↓↓↓negativeサンプル↓↓↓


,Id,Date,Name,Full_text,Judge,Posi_score,Nega_score,Followers,link
0,1210688744660975616,2019-12-28 07:27,comiketofficial,【注意喚起！】寝ている人から財布等を抜き取る犯罪が発生しているという連絡が警察からありました...,negative,61,104,221336,None
1,1210504343675260930,2019-12-27 19:15,comiketofficial,コミックマーケットでは深夜来場は認めておりません。絶対に深夜来場をしないようにお願いします。...,negative,70,99,221336,https://t.co/5WYAt8yOap



↓↓↓fire_tweetサンプル↓↓↓


,Id,Date,Name,Full_text,Judge,Posi_score,Nega_score,Followers,link




csvへの書き出しが完了しました。新規データ数31、全データ数：308
サンプルが0件の場合は、15分後に再度実行すると取得できる場合があります。
fire_tweetは出現率が非常に低いです。


## ２）データセットの前処理
　・正規表現、ストップワード除去など
 
 #### 参考サイト
Pythonで全角・半角記号をまとめて消し去る　http://prpr.hatenablog.jp/entry/2016/11/23/Python%E3%81%A7%E5%85%A8%E8%A7%92%E3%83%BB%E5%8D%8A%E8%A7%92%E8%A8%98%E5%8F%B7%E3%82%92%E3%81%BE%E3%81%A8%E3%82%81%E3%81%A6%E6%B6%88%E3%81%97%E5%8E%BB%E3%82%8B

In [5]:
#必要なツールをインストール(初回のみ実行)
! pip install gensim
! pip install natto-py
! pip install emoji

In [9]:
#ツイートデータを学習用に整形
from natto import MeCab
#import MeCab
import re
import pandas as pd
import pprint
import emoji
import neologdn
import urllib.request
import unicodedata
import string

class For_Model():
    
    def __init__(self, data, columns, out_file, mode, text, similar = None):
        self.mecab = MeCab.Tagger("-Owakati")
        self.data = data
        self.columns = columns
        self.out_file = out_file
        self.mode = mode
        self.text = text
        self.similar = str(similar)

    #データを読み込む
    def Load_tweets(self):        
        df = pd.read_csv(self.data, usecols = self.columns)
        print("読み込んだツイート", df.shape)
        
        #３０w以下のtweet行を削除
        index = []
        for i in range(len(df)):
            text = df.iloc[i, 2]
            text = re.sub('https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
            text = re.sub('http?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
            df.iloc[i, 2] = text
            if len(text) < 30:
                index.append(i)
        df_tweet = df.drop(df.index[index])
        df_tweet = df_tweet.reset_index(drop=True)
        
        #判定用テキストをリストの最後に追加
        tweets = []
        for i in df_tweet[self.text]:
            tweets.append(i)
        if self.similar == None:
            pass
        else:
            tweets.append(self.similar)
        return df_tweet, tweets

    def Stop_Words(self):
        # ストップワードをダウンロード
        url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
        urllib.request.urlretrieve(url, './output/stop_word.txt')

        with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
            stopwords = [word.replace('\n', '') for word in file.readlines()]

        #追加ストップワードを設定（助詞や意味のない平仮名１文字）
        add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                     'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                     'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                     'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                     'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                     'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                     'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                     'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ'
                    ]
        stopwords = stopwords + add_words
        return stopwords

    def Tokenizer(self, text, stopwords):

        words = []
        text = self.mecab.parse(text)
        text = text.split(' ')
        for j in range(len(text)):
            if text[j] not in stopwords:
                words.append(text[j])
        return words

    def remove_emoji(self, text):
        return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)

    #記号削除
    def format_text(self, text):
        text = unicodedata.normalize("NFKC", text)  # 全角記号を半角へ置換
        # 記号を消し去るための魔法のテーブル作成
        table = str.maketrans("", "", string.punctuation  + "「」、。・*`+-|?#!()\[]<>=~/")
        text = text.translate(table)
        return text

    def main(self):
        tweets_num = 0
        stopwords = self.Stop_Words()
        df_tweet, tweets = self.Load_tweets()
        #ツイートを分かち書きしてcsvに出力(モード'a'はデータ追加、モード'w'は新規作成)
        with open('./output/' + self.out_file, self.mode) as f:
            for i in tweets:
                tweets_num += 1
                i = neologdn.normalize(i)
                i = re.sub('\n', "", i)
                i = re.sub(r'[!-~]', "", i)#半角記号,数字,英字
                i = re.sub(r'[︰-＠]', "", i)#全角記号
                i = self.format_text(i)#記号削除
                i = re.sub(r'[【】●ㅅ●Ф☆✩︎♡→←▼①②③④⑤『』ω《》∠∇∩♪∀◞ཀCщ≧≦ ́◤◢■◆★※↑↓〇◯○◎⇒▽◉Θ♫♬〃“”◇ᄉ⊂⊃д°]', "", i)
                #i = re.sub(r'[!-~、。‥…？！〜「」｢｣:：“”【】※♪♩♫♬『』→↓↑《》〈〉[]≧∇≦・゜・●ㅅ●´Д´°ω°•ω•★＊☆♡（）✔Θ∀´∀｀˘ω˘‼бωб￣▽￣◉→←▼①②③④⑤]', "", i)
                i = self.remove_emoji(i)
                i = self.Tokenizer(i, stopwords)
                i = ' '.join(i) #リストを文字列に変換
                i = str(i)
                f.write(i)

        print('CSV出力完了：'+ self.out_file)
        with open('./output/' + self.out_file) as f:
             wakati = f.read()

        print("学習用データに追加したツイート数：", tweets_num)
        print()
        print("分かち書きサンプル\n", wakati)
        return df_tweet


### 前処理の実行

In [10]:
#パラメータの設定

#取得したデータのパス
data = './output/buzz_tweet.csv'
#取得したい列名
columns = ["Followers", "Full_text","Favorite_count","Posi_score", "Nega_score","Judge"]#["text","favorites"]
#出力ファイル名
out_file = "train_buzz.txt"
#学習データの保存モード　'a'：追加／'w'：上書き
mode = 'w'
#ツイートテキストの列を指定
text = "Full_text"
#判定させたいツイート予定文書（類似度確認のため、データセット内にあるツイート文を使用）
similar = "イオンマスク禁止従業員の人嫌がるのわかるわ。\
インフルで出校停止中なんだけど薬効いて体元気だからイオン遊ばせに来た。みたいな事凄く多いんだよ。\
『店員が媒介にならないよう全店でマスク奨励してます。ご理解下さい』\
ってアナウンスされる方が余程良いのでイオンさん、マスク禁止撤回して"

FM = For_Model(data, columns, out_file, mode, text, similar)
df_tweet = FM.main()

読み込んだツイート (308, 6)
CSV出力完了：train_buzz.txt
学習用データに追加したツイート数： 254

分かち書きサンプル
 絶対 断ら ない 評判 病児保育 助成金 下り 赤字 出し 閉鎖 全国 病児保育 赤字 運営 おり 東海 キッズケア 助成金 求め 署名 集め 助成金 下りる あり ませ 社会保障 税金 使わ ませ 
全国 皆さん どうか 皆様 お力 貸し 早く 娘 助け 家族 揃っ 笑顔 クリスマス 新年 迎え 娘 目撃 情報 娘 繋がる 些細なこと 連絡 連絡先 大月 警察 暑 電話 もしくは 最寄り 警察 署 拡散希望 小倉 美咲 
歴史 さんざん 他国 料理 魔 改造 我が国 寿司 ポリス どど 片腹痛い あらゆる 文化 文化 独自 解釈 いい イノベーション 生む 思っ しかし ヘルシンキ 〈 クリスマス トッピング • バナナ 巻き寿司 〉 寛容 試さ いる 
すん ごい フォースリープツイッター 人気 半信半疑 夜 粒 食べ 昨日 めちゃ ぐっすり 眠れ 例える なら 旅行 気持ち良く 遊び 疲れ 夜 睡眠 クオリティ しかも 味 まろやか ミルク チョコ 好み すぎる 目安 一日 粒 摂 れる 
アニメ 期 制作 決定 いたし 期 視聴 応援 くださっ いる 皆様 本当に ありがとう 向け 決意 誓う 空 クロム ゲン 新 カット 実写 実験 による スペシャル 映像 公開 引き続き よろしく お願 石 ドクターストーン 期 
ガンダム まで フレーム 意識 構造 なく モノコック 構造 っぽい 設計 よく 動き まで 無い 可動 エッチ ポーズ 決まり 賛否 アイテム ない 面白い 設計 触る べき 各地 売り切れ 見かけ 買っ 損 ない 
ある日 祖父母 行っ 何故か 大 号泣 何事 理由 聞い 祖父 ぁさ ケーキ あげ たく 秘密 買い 行っ 叱ら 祖母 ぃ 突然 なく 心配 双方 泣き ゃくっててななんなんだこの 可愛い 生き物 衝撃 
年月日 昼 時時 テレビ朝日 ミュージックステーション ウルトラ ミュージカル 刀剣乱舞 刀剣男士 出演 決定 住まい 地域 テレビ番組表 確認 刀 ミュ 
ユニクロ パーカー チャック 音 良い 知っ ほしく チャック 音 使っ 曲 作りました 着心地 普通 
金曜 ジェジ

## ３）予測モデルを生成
　・データセットをDoc２vecで学習<br>

#### 参考サイト
fastTextとDoc2Vecのモデルを作成してニュース記事の多クラス分類の精度を比較する<br> https://qiita.com/kazuki_hayakawa/items/ca5d4735b9514895e197<br>

In [11]:
#Doc2Vecモデルの学習

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

f = open('./output/train_buzz.txt','r')#空白で単語を区切り、改行で文書を区切っているテキストデータ

#１文書ずつ、単語に分割してリストに入れていく[([単語1,単語2,単語3],文書id),...]こんなイメージ
#words：文書に含まれる単語のリスト（単語の重複あり）
# tags：文書の識別子（リストで指定．1つの文書に複数のタグを付与できる）
#fにテキスト データをいれる
trainings = [TaggedDocument(words = data.split(),tags = [i]) for i,data in enumerate(f)]
#print(type(trainings))
print("Doc２vec文書ベクトル用モデルに学習させたツイート数",len(trainings))
# print(trainings[:20])

#文書ベクトル用ツイートテキストの学習
model = Doc2Vec(
    documents= trainings, 
    dm = 1, 
    vector_size=300, 
    window=10, 
    alpha = 0.05, 
    min_count=1, 
    sample = 0, 
    workers=4, 
    epochs = 50
)

#出力用ディレクトリ作成（存在しない場合のみ）
def Make_Dir():
    new_dir_path = 'model'
    try:
        os.makedirs(new_dir_path)
    except FileExistsError:
        pass

# モデルのセーブ
Make_Dir()
model.save("./model/doc2vec.model")

# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load('./model/doc2vec.model')

Doc２vec文書ベクトル用モデルに学習させたツイート数 254


## ４）ツイート予定文章のネガポジ予測を返す
　・データセットから、入力しておいたツイート予定文書と似ている文書を探す<br>
・ネガポジスコア付きで、類似ツイート上位１０個を返す<br>
#### 結果：成功。入力文書と同じツイート文が類似度１位に。ネガポジもデータズレなく表示できた

In [13]:
#類似判定と類似している上位10件の文書を出力

top10 = m.docvecs.most_similar(len(trainings) - 1)

print("=========== 判定したいツイート ===========\n")
print(similar)

print()
print("======= 類似度上位１０（全{}ツイート中） =======".format(len(trainings)))
print()
for i in range(len(top10)):
    score = top10[i]
    index = int(score[0])
    similar_score = score[1]
    tweet = df_tweet["Full_text"]
    judge = df_tweet["Judge"]
    posi_score = df_tweet["Posi_score"]
    nega_score = df_tweet["Nega_score"]
    print("…………　類似ツイート{}位：類似度 {:.4g}　…………".format((i+1), similar_score))
    print()
    print(tweet[index])
    print()
    print("【極性】：", judge[index])
    print("posi_score：",posi_score[index], "／", "nega_score：", nega_score[index])

    print()



=========== 判定したいツイート ===========

イオンマスク禁止従業員の人嫌がるのわかるわ。インフルで出校停止中なんだけど薬効いて体元気だからイオン遊ばせに来た。みたいな事凄く多いんだよ。『店員が媒介にならないよう全店でマスク奨励してます。ご理解下さい』ってアナウンスされる方が余程良いのでイオンさん、マスク禁止撤回して

======= 類似度上位１０（全254ツイート中） =======

…………　類似ツイート1位：類似度 0.9934　…………

イオンマスク禁止従業員の人嫌がるのわかるわ。

インフルで出校停止中なんだけど薬効いて体元気だからイオン遊ばせに来た。
みたいな事凄く多いんだよ。

『店員が媒介にならないよう全店でマスク奨励してます。ご理解下さい』
ってアナウンスされる方が余程良いのでイオンさん、マスク禁止撤回して

【極性】： negative
posi_score： 219.0 ／ nega_score： 257.0

…………　類似ツイート2位：類似度 0.9563　…………

マスクせずに咳してる人見ると、
「感染者だ！撃ち殺せ！」
「ですが！アイツはまだ人間じゃないですか！」
「馬鹿野郎！ここで殺らなきゃ、俺達の大事な奴まで感染者にされる可能性だってあるんだ！迷うな！引き金を引け！」
「畜生！感染者め！」
みたいな気持ちになる。マスクしやがり下さい。

【極性】： negative
posi_score： 119.0 ／ nega_score： 213.0

…………　類似ツイート3位：類似度 0.9296　…………

お客様に言いたいのは、店員にマスクをしてほしくないなら、お客様がしてくれと言うことと、ポイントカードの有無も答えず、キャッシュ決済で払も無言でスマホだけレジ台に置く、会計時イヤホン外さない、こういう人は、「何言ってるか聞こえない」なんて言う権利ないからな！
＃マスク禁止

【極性】： negative
posi_score： 105.0 ／ nega_score： 118.0

…………　類似ツイート4位：類似度 0.9189　…………

詩織さんに関して「もう女と酒を飲まない方が良い。これで少子化が進みそう」とリプしたおじさん、限界過ぎ。
女と酒飲む＝性行為という認識が誤りだし、合意の無い性行為＝強

# その他試みたこと
断念、または精度が全く良くない。覚書として記録

## １）文章ベクトルを特徴量としたネガポジ予測モデル
　・文章ベクトルとフォロワー数を特徴量X、ネガポジスコアを目的変数yとしたデータを学習<br>
　・文章ベクトルはDoc２vecとTf-idfの２種を作成<br>
　・ツイート予定文書を入力してネガポジスコアを予測する<br>
　・試した予測モデル<br>
　・MultiOutputRegressor、SVRのrbf と　SVRの線形、lightgbm、ランダムフォレスト<br>
#### 結果：精度が低すぎて断念<br>

### Doc2vecで文章ベクトル取得

In [30]:
#Doc2vecでベクトル化
from natto import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer

df_buzz = pd.read_csv('./output/buzz_tweet.csv',
                      usecols = ["Full_text", "Posi_score", "Nega_score", "Followers"])
#.to_csv('./output/for_training.csv', mode = "a", index = False, header = None)
#pd.read_csv('./output/fire_buzz_tweet.csv', usecols = ["Full_text", "Judge", "Sentiment"]).to_csv('./output/for_training.csv', mode = "a", index = False, header = None)
print("ベクトル化するセンチメントスコア付きデータ数：", len(df_buzz))
display(df_buzz.head())

#doc2vecでベクトル化
for_training = df_buzz['Full_text']
#print(for_training)
vector_tweet = []
for i in for_training:
    i = m.infer_vector(i)
    vector_tweet.append(i)

df_vector = pd.DataFrame(data = vector_tweet)

# print("Doc2vecベクトル")
# display(df_vector.head())

ベクトル化するセンチメントスコア付きデータ数： 308


,Followers,Full_text,Nega_score,Posi_score
0,6800.0,君、すごい食い方やな\n https://t.co/yRTGvd43wT,31.0,75.0
1,1564.0,絶対断らないと評判の病児保育室、助成金下りず2億円の赤字を出し閉鎖\n\n全国に2886カ所...,148.0,168.0
2,2217.0,全国の皆さんへ\nどうか皆様のお力を貸してください。\n\n１日も早く娘を助けたいです。\n...,50.0,129.0
3,3756.0,o0( 歴史上、さんざん他国の料理を魔改造してきた我が国が「寿司ポリス」などどは片腹痛い！あ...,85.0,137.0
4,7363.0,百合関係図です。 https://t.co/hZxIYOsSag,58.0,82.0


### Tf-idfでベクトル取得
#### 参考サイト

機械学習_サポートベクターマシーン_pythonで実装<br>
https://dev.classmethod.jp/machine-learning/2017ad_20171214_svm_python/<br>
Tf-idfベクトルってなんだ？<br> https://qiita.com/MasatoTsutsumi/items/5b0a140b1ecbdd0396e1

In [31]:
# 2-1.tf-idf計算
from sklearn.feature_extraction.text import TfidfVectorizer

def Stop_Words():
    # ストップワードをダウンロード
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    urllib.request.urlretrieve(url, './output/stop_word.txt')

    with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
        stopwords = [word.replace('\n', '') for word in file.readlines()]

    #追加ストップワードを設定（助詞や意味のない平仮名１文字）
    add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                 'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                 'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                 'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                 'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                 'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                 'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                 'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ'
                ]
    stopwords = stopwords + add_words
    return stopwords

stopwords = Stop_Words()
tfidfv = TfidfVectorizer(lowercase=True, stop_words=stopwords) # stop words処理
 
tfv_vector_fit = tfidfv.fit(for_training)
tfv_vector = tfv_vector_fit.transform(for_training)
print(tfv_vector.shape) 

# 2-2.次元削減(「lsa」を使って次元削減を行う)
from sklearn.decomposition import TruncatedSVD

# 2-2-1.パラメータの調整
list_n_comp = [5,10,50,100,500,1000] # 特徴量を何個に削減するか、というパラメータです。できるだけ情報量を欠損しないで、かつ次元数は少なくしたいですね。
for i in list_n_comp:
    lsa = TruncatedSVD(n_components=i,n_iter=5, random_state = 0)
    lsa.fit(tfv_vector) 
    tfv_vector_lsa = lsa.transform(tfv_vector)
    print('次元削減後の特徴量が{0}の時の説明できる分散の割合合計は{1}です'.format(i,round((sum(lsa.explained_variance_ratio_)),2)))

# 2-2-2.次元削減した状態のデータを作成
# 上記で確認した「n_components」に指定した上で、次元削減（特徴抽出）を行う
lsa = TruncatedSVD(n_components=1000, n_iter=5, random_state = 0) # 今回は次元数を1000に指定
lsa.fit(tfv_vector)
X_tf = lsa.transform(tfv_vector)
# print()
# print("次元削減後Tf-idfベクトル\n", X_tf.shape)
# print(X_tf)

(308, 2038)
次元削減後の特徴量が5の時の説明できる分散の割合合計は0.04です
次元削減後の特徴量が10の時の説明できる分散の割合合計は0.06です
次元削減後の特徴量が50の時の説明できる分散の割合合計は0.2です
次元削減後の特徴量が100の時の説明できる分散の割合合計は0.36です
次元削減後の特徴量が500の時の説明できる分散の割合合計は1.0です
次元削減後の特徴量が1000の時の説明できる分散の割合合計は1.0です


In [23]:
#X、yデータを作成

#Doc2vecのベクトルデータ
print("欠損値削除前データ", df_buzz.shape)
print()

#文書ベクトルを含んだdf
df_buzz_vec = pd.concat([df_buzz, df_vector], axis=1)
df_buzz_vec = df_buzz_vec.dropna(subset = ["Followers"])#欠損値行削除
df_buzz_vec = df_buzz_vec.drop([ "Full_text", "Nega_score", "Posi_score"], axis=1)
X = df_buzz_vec.values
print("Doc2vecベクトル")
print("X.shape", X.shape)
display(df_buzz_vec.head())

#tf-idfのベクトルデータ
tf_df = pd.DataFrame(data = X_tf)
tf_df = pd.concat([df_buzz, tf_df], axis=1)
tf_df = tf_df.dropna(subset = ["Followers"])#欠損値行削除
tf_df = tf_df.drop([ "Full_text", "Nega_score", "Posi_score"], axis=1)
X_tf_idf = tf_df.values
print("Tf-idfベクトル")
print("X_tf_idf.shape", tf_df.shape)
display(tf_df.head())

#yデータ作成
df_buzz = df_buzz.dropna(subset = ["Followers"])#y用に"Followers"の欠損行削除
y = df_buzz.loc[:,['Posi_score', 'Nega_score']]#できればDateも特徴量に入れたい
y_p = df_buzz['Posi_score']
y_n = df_buzz['Nega_score']
y = y.values
print()
print("y.shape", y.shape)
y_p = y_p.values
y_n = y_n.values

欠損値削除前データ (307, 4)

Doc2vecベクトル
X.shape (307, 301)


,Followers,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,6800.0,0.013712,0.001771,-0.011958,-0.010011,0.030667,0.018747,-0.013943,0.009593,-0.002913,...,0.019474,0.003289,-0.013615,0.001718,0.029068,0.021951,0.015445,-0.005926,-0.009584,-0.024975
1,1564.0,0.021855,0.000419,-0.009026,-0.024573,0.014405,0.030817,-0.010246,0.027657,-0.004478,...,0.023794,0.003821,-0.010664,0.002451,0.012792,0.008937,0.004685,0.006093,-0.023806,-0.024231
2,2217.0,-0.000127,-0.006724,-0.008196,-0.011883,0.033857,0.006988,-0.013092,0.004137,0.002084,...,0.014613,0.004554,-0.003018,0.014777,0.030124,0.030675,0.016303,-0.014497,0.002742,-0.015558
3,3756.0,0.006243,0.006586,0.010837,0.006348,0.013797,0.000452,-0.012791,-0.001024,-0.005264,...,0.007500,0.006826,0.003031,0.011547,0.018484,0.017857,0.005455,0.002538,-0.012708,0.001583
4,7363.0,0.004367,-0.006778,-0.006213,-0.014475,0.028134,0.008578,-0.012496,0.006429,0.001018,...,0.012125,0.004307,-0.006882,0.009480,0.022597,0.028421,0.015285,-0.008094,-0.002776,-0.009579


Tf-idfベクトル
X_tf_idf.shape (307, 309)


,Followers,0,1,2,3,4,5,6,7,8,...,298,299,300,301,302,303,304,305,306,307
0,6800.0,0.289868,-0.000427,0.000041,-0.002769,-1.424757e-16,-0.005925,-0.011368,-0.014004,0.000940,...,-5.008939e-07,-0.000008,-0.000028,0.000955,-0.003888,0.000314,-0.000107,0.000044,2.827210e-18,-0.000027
1,1564.0,0.172859,-0.000260,0.000026,-0.001753,-6.100152e-16,-0.003930,-0.007584,-0.009409,0.000632,...,-2.420091e-07,-0.000004,-0.000015,0.000496,-0.002029,0.000166,-0.000057,0.000024,-1.025210e-16,-0.000015
2,2217.0,0.108238,-0.000168,0.000017,-0.001208,3.493364e-16,-0.002950,-0.005759,-0.007248,0.000488,...,-1.183614e-07,-0.000002,-0.000008,0.000261,-0.001073,0.000089,-0.000031,0.000013,-5.549815e-17,-0.000008
3,3756.0,0.110438,-0.000167,0.000017,-0.001143,2.116256e-17,-0.002606,-0.005039,-0.006268,0.000421,...,-1.456683e-07,-0.000003,-0.000009,0.000304,-0.001246,0.000102,-0.000036,0.000015,5.629822e-17,-0.000009
4,7363.0,0.289868,-0.000427,0.000041,-0.002769,1.624278e-18,-0.005925,-0.011368,-0.014004,0.000940,...,-5.008939e-07,-0.000008,-0.000028,0.000955,-0.003888,0.000314,-0.000107,0.000044,8.386501e-17,-0.000027



y.shape (307, 2)


### MultiOutputRegressorで複数の回帰¶
#### 結果：D2vベクトルよりTf-idfがややマシ

In [32]:
#MultiOutputRegressorで複数の回帰

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

#D2vベクトル
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=0)

#X, y = make_regression(n_samples=10, n_targets=3, random_state=1)
MOR = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train)
y_pred = MOR.predict(X_test)
score = MOR.score(X_test, y_test)

print("正解\n", y_test)
print()
print(y_pred)
print("R ^ 2_score(1に近いほど良い）：", score)
print()

#X_tf tf-idfベクトルを使った予測
X_train, X_test, y_train, y_test = train_test_split(
    X_tf_idf, y, train_size=0.7, test_size=0.3, random_state=0)

MOR = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train)
y_pred = MOR.predict(X_test)
score = MOR.score(X_test, y_test)
print(y_pred)
print("R ^ 2_score(1に近いほど良い）：", score)

正解
 [[ 171.  214.]
 [  65.   79.]
 [  87.   11.]
 [  91.   89.]
 [  95.   57.]
 [  48.   37.]
 [  20.   16.]
 [  18.   46.]
 [  59.   54.]
 [  64.  143.]
 [ 118.   36.]
 [  76.   46.]
 [  35.   23.]
 [  52.   15.]
 [  44.   30.]
 [  61.   46.]
 [  25.   27.]
 [  65.   43.]
 [  77.    5.]
 [  88.   99.]
 [ 122.  137.]
 [ 160.  144.]
 [  63.   29.]
 [ 118.  227.]
 [ 114.  173.]
 [  48.   39.]
 [ 114.   43.]
 [  40.   63.]
 [ 247.   91.]
 [ 119.  150.]
 [  87.   28.]
 [ 124.  102.]
 [  71.   58.]
 [  37.   49.]
 [  73.   59.]
 [ 128.  197.]
 [ 127.   89.]
 [ 180.   46.]
 [ 283.  219.]
 [  69.  137.]
 [ 102.   81.]
 [  38.   42.]
 [  45.   35.]
 [ 210.  169.]
 [ 186.  106.]
 [ 156.   20.]
 [  50.   19.]
 [ 146.   61.]
 [  92.   57.]
 [ 102.   29.]
 [  90.   19.]
 [ 197.    9.]
 [ 219.  257.]
 [ 117.   26.]
 [ 116.  142.]
 [ 179.   41.]
 [  92.   76.]
 [ 166.  107.]
 [ 190.  157.]
 [ 100.   79.]
 [ 149.   50.]
 [ 174.  134.]
 [  97.   78.]
 [ 141.   39.]
 [ 171.  121.]
 [ 205.   45.]
 [  90

### SVRのrbf と　SVRの線形で予測
#### 結果：予測値が全くダメ

In [36]:
#ポジ、ネガ別々で予測する
#SVRのrbf と　SVRの線形

import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt

#D2vベクトル(ポジのみ)
X_train, X_test, y_p_train, y_p_test = train_test_split(
    X, y_p, train_size=0.7, test_size=0.3, random_state=0)

svr_rbf = SVR(kernel='rbf', C=1, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1)
y_rbf = svr_rbf.fit(X_train, y_p_train)
y_lin = svr_lin.fit(X_train, y_p_train)

pred_rbf = svr_rbf.predict(X_test)
pred_lin = svr_lin.predict(X_test)

#精度

# 相関係数計算
rbf_corr = np.corrcoef(y_p_test, pred_rbf)[0, 1]
lin_corr = np.corrcoef(y_p_test, pred_lin)[0, 1]

# RMSEを計算（０に近いほど良い）
rbf_rmse = sqrt(mean_squared_error(pred_rbf, y_p_test))
lin_rmse = sqrt(mean_squared_error(pred_lin, y_p_test))

print("RBF: RMSE（０に近いほど良い） {} ".format(rbf_rmse))
print("Linear: RMSE（０に近いほど良い） {}" .format(lin_rmse))
print()
print("正解", y_p_test)
print("rbf推定", pred_rbf)
print("lin推定", pred_lin)


RBF: RMSE（０に近いほど良い） 57.128553013108764 
Linear: RMSE（０に近いほど良い） 12268.253334406958
正解 [ 171.   65.   87.   91.   95.   48.   20.   18.   59.   64.  118.   76.
   35.   52.   44.   61.   25.   65.   77.   88.  122.  160.   63.  118.
  114.   48.  114.   40.  247.  119.   87.  124.   71.   37.   73.  128.
  127.  180.  283.   69.  102.   38.   45.  210.  186.  156.   50.  146.
   92.  102.   90.  197.  219.  117.  116.  179.   92.  166.  190.  100.
  149.  174.   97.  141.  171.  205.   90.  244.   73.   83.  120.   53.
   80.   93.  147.  100.   31.   94.   46.  162.  165.  116.  116.   93.
   82.  126.   55.   84.  112.   40.  141.   39.   79.]
rbf推定 [ 91.37999996  91.38000001  90.38265887  91.38000005  91.38000001
  91.38000001  90.38915892  90.97596795  91.38000001  91.37995499
  91.38000001  91.38019787  91.38000001  91.38000001  91.40725473
  91.38000001  91.38000001  91.38000001  91.38000001  91.38000001
  91.38000001  91.38000001  91.38000001  91.38000001  91.38000001
  91.3800000

### lightgbm
#### 参考サイト

Mercari Price Challenge -機械学習を使ったメルカリの価格予測 Ridge回帰 LightGBM

http://rautaku.hatenablog.com/entry/2017/12/22/195649

#### 結果：RMSEが0には程遠い

In [37]:
#必要なツールをインストール(初回のみ実行)
! pip install lightgbm

In [38]:
#LightGBM を使った回帰予測(D2Vベクトル)

import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

def main():
    #D2vベクトル(ポジのみ)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_p, train_size=0.7, test_size=0.3, random_state=0)

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # LightGBM のハイパーパラメータ
    lgbm_params = {
        # 回帰問題
        'objective': 'regression',
        # RMSE (平均二乗誤差平方根) の最小化を目指す
        'metric': 'rmse',
    }

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train, 
                      valid_sets=lgb_eval, num_boost_round=8000, 
                      early_stopping_rounds=5000, verbose_eval=500)

    # テストデータを予測する
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # RMSE を計算する
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("RMSE（０に近いほど良い）", rmse)

if __name__ == '__main__':
    main()

Training until validation scores don't improve for 5000 rounds
[500]	valid_0's rmse: 60.8761
[1000]	valid_0's rmse: 61.0528
[1500]	valid_0's rmse: 61.0861
[2000]	valid_0's rmse: 61.0921
[2500]	valid_0's rmse: 61.0933
[3000]	valid_0's rmse: 61.0935
[3500]	valid_0's rmse: 61.0936
[4000]	valid_0's rmse: 61.0936
[4500]	valid_0's rmse: 61.0936
[5000]	valid_0's rmse: 61.0936
Early stopping, best iteration is:
[6]	valid_0's rmse: 54.7642
RMSE（０に近いほど良い） 54.7641918682


In [39]:
#LightGBM を使った回帰予測（Tfーidfベクトル）

def main():

    #X_tf tf-idfベクトルを使った予測(ポジのみ)
    X_train, X_test, y_train, y_test = train_test_split(
        X_tf_idf, y_p, train_size=0.7, test_size=0.3, random_state=0)

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # LightGBM のハイパーパラメータ
    lgbm_params = {
        # 回帰問題
        'objective': 'regression',
        # RMSE (平均二乗誤差平方根) の最小化を目指す
        'metric': 'rmse',
    }
    
    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train, 
                      valid_sets=lgb_eval, num_boost_round=8000, 
                      early_stopping_rounds=5000, verbose_eval=500)
#     model = lgb.LGBMRegressor()
#     model.fit(X_train, y_train)

    # テストデータを予測する
    y_pred = model.predict(X_test)

    # RMSE を計算する
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("RMSE（０に近いほど良い）",rmse)


if __name__ == '__main__':
    main()

Training until validation scores don't improve for 5000 rounds
[500]	valid_0's rmse: 53.7358
[1000]	valid_0's rmse: 53.7507
[1500]	valid_0's rmse: 53.7513
[2000]	valid_0's rmse: 53.751
[2500]	valid_0's rmse: 53.751
[3000]	valid_0's rmse: 53.7511
[3500]	valid_0's rmse: 53.7511
[4000]	valid_0's rmse: 53.7511
[4500]	valid_0's rmse: 53.7511
[5000]	valid_0's rmse: 53.7511
Early stopping, best iteration is:
[64]	valid_0's rmse: 53.5511
RMSE（０に近いほど良い） 53.5511129593


### ランダムフォレスト

In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

#D2vベクトル
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=0)
# ランダムフォレスト回帰オブジェクト生成
rfr = RandomForestRegressor(n_estimators=100)
# 学習の実行
rfr.fit(X_train, y_train)
# テストデータで予測実行
predict_y = rfr.predict(X_test)
# R2決定係数で評価
r2_score = r2_score(y_test, predict_y)
print("R^2(1に近いほど良い）:", r2_score)


R^2(1に近いほど良い）: -0.147252936866


## ２）ツイッターAPI制限への挑戦（データセットの拡大）
　・古いツイートを大量取得できるパッケージを発見（通常は１週間程度しか遡れない）<br>
#### 結果：取得データから反応ツイートの取得を試みたができなかった<br>

### GetOldTweets3 0.0.11
古いツイートをトークン申請なしで大量取得できるパッケージ<br>
https://pypi.org/project/GetOldTweets3/

In [ ]:
#必要なツールをインストール(初回のみ実行)
! pip install GetOldTweets3

In [ ]:
#指定日のトップツイートを取得、'./output/toptweets.csv'に保存
! GetOldTweets3 --lang ja  --toptweets  --querysearch "" --since 2019-2-10 --until 2019-2-11 --output './output/toptweets.csv'
